# Hazard Data Testing - FEMA Flood & USGS Earthquake APIs
Test the new FEMA Flood and USGS Earthquake API clients

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from loguru import logger

from src.api_clients import FEMAFloodClient, USGSEarthquakeClient
from src.pipeline.data_enrichment import DataEnrichmentPipeline

# Configure logger
logger.add("../logs/hazard_testing.log", rotation="10 MB")

print("✓ Imports successful")

## Test 1: FEMA Flood Client

In [ ]:
# Initialize FEMA client
fema = FEMAFloodClient()
print("✓ FEMA Flood Client initialized")

In [ ]:
# Test locations with different flood risks
test_locations = [
    {'name': 'New Orleans (High Risk)', 'lat': 29.9511, 'lon': -90.0715},
    {'name': 'Houston (Moderate Risk)', 'lat': 29.7604, 'lon': -95.3698},
    {'name': 'Denver (Low Risk)', 'lat': 39.7392, 'lon': -104.9903},
    {'name': 'San Francisco', 'lat': 37.7749, 'lon': -122.4194},
]

print("Testing FEMA Flood API with diverse locations:\n")

for loc in test_locations:
    print(f"\n{'='*60}")
    print(f"Location: {loc['name']}")
    print(f"Coordinates: ({loc['lat']}, {loc['lon']})")
    print(f"{'='*60}")
    
    flood_data = fema.get_flood_zone(loc['lat'], loc['lon'])
    
    print(f"\nFlood Zone: {flood_data['flood_zone']}")
    print(f"Risk Score: {flood_data['flood_risk_score']:.2f}")
    print(f"Risk Category: {fema.get_flood_risk_category(flood_data['flood_zone'])}")
    print(f"Base Flood Elevation: {flood_data.get('base_flood_elevation', 'N/A')}")
    print(f"Data Source: {flood_data['data_source']}")

In [ ]:
# Test enrichment features
print("\n" + "="*60)
print("FEMA ENRICHMENT FEATURES TEST")
print("="*60)

lat, lon = 29.9511, -90.0715  # New Orleans
features = fema.get_enrichment_features(lat, lon)

print(f"\nEnrichment features for New Orleans:")
for key, value in features.items():
    print(f"  {key}: {value}")

## Test 2: USGS Earthquake Client

In [ ]:
# Initialize USGS client
usgs = USGSEarthquakeClient()
print("✓ USGS Earthquake Client initialized")

In [ ]:
# Test locations with different seismic risks
seismic_locations = [
    {'name': 'San Francisco (High Risk)', 'lat': 37.7749, 'lon': -122.4194},
    {'name': 'Los Angeles (High Risk)', 'lat': 34.0522, 'lon': -118.2437},
    {'name': 'Seattle (Moderate Risk)', 'lat': 47.6062, 'lon': -122.3321},
    {'name': 'New York (Low Risk)', 'lat': 40.7128, 'lon': -74.0060},
    {'name': 'Kansas City (Minimal Risk)', 'lat': 39.0997, 'lon': -94.5786},
]

print("Testing USGS Earthquake API with diverse locations:\n")

for loc in seismic_locations:
    print(f"\n{'='*60}")
    print(f"Location: {loc['name']}")
    print(f"Coordinates: ({loc['lat']}, {loc['lon']})")
    print(f"{'='*60}")
    
    # Get earthquake statistics
    stats = usgs.get_earthquake_statistics(loc['lat'], loc['lon'], radius_km=50, years=10)
    
    print(f"\nEarthquake Statistics (10 years, 50km radius):")
    print(f"  Total earthquakes: {stats['total_count']}")
    print(f"  Magnitude 3.0+: {stats['count_magnitude_3plus']}")
    print(f"  Magnitude 5.0+: {stats['count_magnitude_5plus']}")
    print(f"  Max magnitude: {stats['max_magnitude']:.1f}")
    print(f"  Avg magnitude: {stats['avg_magnitude']:.2f}")
    print(f"  Per year: {stats['earthquakes_per_year']:.1f}")
    
    # Get hazard score
    hazard_score = usgs.get_seismic_hazard_score(loc['lat'], loc['lon'])
    risk_category = usgs.get_seismic_risk_category(hazard_score)
    
    print(f"\nSeismic Hazard Score: {hazard_score:.3f}")
    print(f"Risk Category: {risk_category}")

In [ ]:
# Get detailed earthquake list for San Francisco
print("\n" + "="*60)
print("DETAILED EARTHQUAKE DATA - SAN FRANCISCO")
print("="*60)

earthquakes = usgs.get_earthquakes_in_area(
    lat=37.7749,
    lon=-122.4194,
    radius_km=50,
    min_magnitude=4.0
)

print(f"\nFound {len(earthquakes)} earthquakes (magnitude >= 4.0)\n")

if earthquakes:
    # Show top 5 largest
    print("Top 5 Largest Earthquakes:")
    for i, eq in enumerate(earthquakes[:5], 1):
        print(f"\n{i}. Magnitude {eq['magnitude']:.1f}")
        print(f"   Location: {eq['place']}")
        print(f"   Date: {eq['time_str']}")
        print(f"   Depth: {eq['depth_km']:.1f} km")

In [ ]:
# Test enrichment features
print("\n" + "="*60)
print("USGS ENRICHMENT FEATURES TEST")
print("="*60)

lat, lon = 37.7749, -122.4194  # San Francisco
features = usgs.get_enrichment_features(lat, lon)

print(f"\nEnrichment features for San Francisco:")
for key, value in features.items():
    print(f"  {key}: {value}")

## Test 3: Combined Hazard Analysis

In [ ]:
# Analyze combined hazard risk for multiple cities
print("\n" + "="*60)
print("COMBINED HAZARD RISK ANALYSIS")
print("="*60)

cities = [
    {'name': 'San Francisco', 'lat': 37.7749, 'lon': -122.4194},
    {'name': 'New Orleans', 'lat': 29.9511, 'lon': -90.0715},
    {'name': 'Miami', 'lat': 25.7617, 'lon': -80.1918},
    {'name': 'Denver', 'lat': 39.7392, 'lon': -104.9903},
]

results = []

for city in cities:
    # Get flood data
    flood_data = fema.get_flood_zone(city['lat'], city['lon'])
    
    # Get earthquake data
    eq_score = usgs.get_seismic_hazard_score(city['lat'], city['lon'])
    
    # Calculate composite risk
    composite_risk = (flood_data['flood_risk_score'] + eq_score) / 2
    
    results.append({
        'City': city['name'],
        'Flood Zone': flood_data['flood_zone'],
        'Flood Risk': flood_data['flood_risk_score'],
        'Seismic Risk': eq_score,
        'Composite Risk': composite_risk
    })

df_hazards = pd.DataFrame(results)
df_hazards = df_hazards.sort_values('Composite Risk', ascending=False)

print("\nHazard Risk Comparison:")
print(df_hazards.to_string(index=False))

## Test 4: Unified Enrichment Pipeline

In [ ]:
# Initialize enrichment pipeline
# Note: Census and OpenCelliD require API keys from .env file
pipeline = DataEnrichmentPipeline()

print("✓ Enrichment Pipeline initialized")
print("\nFeature Summary:")
summary = pipeline.get_feature_summary()
print(f"Total data sources: {summary['total_sources']}")
for source, info in summary['sources'].items():
    if info['available']:
        print(f"  ✓ {source}: {len(info['features'])} features")
    else:
        print(f"  ✗ {source}: {info.get('reason', 'Not available')}")

In [ ]:
# Test enrichment on sample sites
sample_sites = [
    {'site_id': 'SITE001', 'latitude': 37.7749, 'longitude': -122.4194, 'city': 'San Francisco'},
    {'site_id': 'SITE002', 'latitude': 29.9511, 'longitude': -90.0715, 'city': 'New Orleans'},
    {'site_id': 'SITE003', 'latitude': 40.7128, 'longitude': -74.0060, 'city': 'New York'},
]

print("\n" + "="*60)
print("ENRICHING SAMPLE SITES")
print("="*60)

enriched_df = pipeline.enrich_batch(sample_sites, show_progress=True)

print(f"\nEnriched {len(enriched_df)} sites")
print(f"Total features per site: {len(enriched_df.columns)}")

# Show hazard-related columns
hazard_cols = [col for col in enriched_df.columns if 'fema' in col or 'usgs' in col or 'risk' in col]
print(f"\nHazard-related features ({len(hazard_cols)}):")
print(enriched_df[['site_id', 'city'] + hazard_cols].to_string(index=False))

In [ ]:
# Show all enriched features for one site
print("\n" + "="*60)
print("COMPLETE FEATURE SET - San Francisco Site")
print("="*60)

sf_site = enriched_df[enriched_df['city'] == 'San Francisco'].iloc[0]

print("\nAll enriched features:")
for col in enriched_df.columns:
    value = sf_site[col]
    if pd.notna(value):
        if isinstance(value, float):
            print(f"  {col}: {value:.4f}")
        else:
            print(f"  {col}: {value}")

## Test 5: Batch Processing Performance

In [ ]:
import time

# Test batch processing with 10 locations
test_batch = [
    {'site_id': f'SITE{i:03d}', 'latitude': 37.7749 + i*0.1, 'longitude': -122.4194 + i*0.1}
    for i in range(10)
]

print("\n" + "="*60)
print("BATCH PROCESSING PERFORMANCE TEST")
print("="*60)

start_time = time.time()
enriched_batch = pipeline.enrich_batch(test_batch, show_progress=True)
elapsed_time = time.time() - start_time

print(f"\nProcessed {len(enriched_batch)} sites in {elapsed_time:.2f} seconds")
print(f"Average time per site: {elapsed_time/len(enriched_batch):.2f} seconds")
print(f"Total features per site: {len(enriched_batch.columns)}")

## Summary

This notebook tested:
1. ✅ FEMA Flood API - Flood zone lookup and risk scoring
2. ✅ USGS Earthquake API - Seismic hazard analysis
3. ✅ Combined hazard risk analysis
4. ✅ Unified enrichment pipeline with all data sources
5. ✅ Batch processing performance

### Key Features Added:
- **FEMA Flood:** 6 features (zone, risk score, category, elevation, etc.)
- **USGS Earthquake:** 9 features (counts, magnitudes, hazard score, etc.)
- **Derived:** 3 features (composite risk, market opportunity, etc.)

### Next Steps:
- Export enriched data for modeling
- Analyze feature correlations
- Build cross-sectional dataset
- Develop fair rent forecasting models